## Building a CNN
This notebook goes through the steps of piecing together a CNN to tackle the CIFAR10 dataset. This is the practical application of the theory covered in Chapter 6: Intro to Convolutional Neural Networks. Although we are using Keras and TensorFlow in this example other frameworks are very similar.

In [ ]:
import tensorflow as tf

In [ ]:
N_CLASSES = 10 # There are 10 classes in the CIFAR10 dataset
data_format = 'channels_last'

So we need to build a CNN which takes a color 32x32 image in and outputs a ten dimensional vector where the position in the vector that corresponds to the class of the image is 1 and the rest are 0.

We care going to build our CNN layer by layer and take a look at what the output is of each successsive layers
Our input is to be data from the CIFAR10 dataset so the dimnesions will be 32x32x3 (Width x Height x RGB)
First we will add a convolution layer with 50 filters and each filter will have a kernel size of 3x3. We will also set the activation as ReLU

In [ ]:
model_input = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])

In [ ]:
conv1 = tf.layers.conv2d(model_input, 
                             filters=50, 
                             kernel_size=(3, 3), 
                             padding='valid', 
                             data_format=data_format)
print(conv1)

The first dimensions reported in the shape of the above tensor would be the batch size but since we haven't fed that in the model yet it has a question mark. The next two values are the width and height which is reduced by 2 due to the size of the kernel filters, the stride and the padding. If we were to set the padding argument to "same" rather than "valid" the dimensions of the output would remain the same as the input. Play around with the kernel size stride and padding so that you get a feel as to what it does. The reasons behind this were exaplained earlier on in the chapter "Intro to Convolutional Neural Networks". The final dimension is the number of filters/kernels.

Let's add a second convolution layer with 25 filters and set the padding to "same".

In [ ]:
relu1 = tf.nn.relu(conv1)

In [ ]:
conv2 = tf.layers.conv2d(relu1, 
                             filters=25, 
                             kernel_size=(3, 3), 
                             padding='same', 
                             data_format=data_format)
print(conv2)

Now we see that the last dimension is 25 matching the number of filters we specified for this layer.

Next we add a max pooling layer. 

In [ ]:
pool1 = tf.layers.max_pooling2d(conv2, 
                                    pool_size=(2, 2), 
                                    strides=(2, 2), 
                                    padding='valid', 
                                    data_format=data_format)
print(pool1)

In [ ]:
relu2 = tf.nn.relu(pool1)
print(relu2)

As you can see the effect it has on the dimensions of the input is to decrease the width and height by half. This is because we have chosen a stride of (2,2). With a stride of (3,3) the width and weight would have been a third of the size of the input. It is common practice for the dimensions of the pooling size and strides to be the same due to the nature of the max pooling layer. For different pooling layers this isn't always the case.

Let's add a dropout layer. Here we specify a dropout rate of 0.25 which means that randomly during each batch 25% of the connections will be droped. What this means is that the values in those connections are zeroed out randomly for each batch. The purpose of dropout is to prevent overfitting and help with generalisation. 

In [ ]:
drop1 = tf.layers.dropout(relu2, 0.25)

Notice that this does not affect the size of our output

In [ ]:
print(drop1)

Now we are going to go to what can be thought of as the classification layers of the network. First we need to flatten out everything into one long vector and add a denseley connected layer.

In [ ]:
flatten = tf.reshape(relu2, shape=[-1, 50*16*16])
fc1 = tf.layers.dense(flatten, 512, activation=tf.nn.relu)

Now the size of the output from the denseley connecte layer will be the batch size and the number of units in the denseley connected layer.

In [ ]:
print(fc1)

Next let's add another dropout layer and the final activation layer. The output of the final layer will be the same number as the number of classes we have. Therefore when the model detects a certain class will expect all the rest of the values to be 0 and the desired class to have a value of 1

In [ ]:
drop2 = tf.layers.dropout(fc1, 0.5)
logits = tf.layers.dense(fc1, N_CLASSES, name='output')
print(logits)

This covers the basic types of layers you would have in a convolutional neural network. 